In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
%load_ext autoreload
%autoreload 2
#useful for displaying wide data frames
pd.set_option('display.max_columns', 50)

In [2]:
#load the data into Pandas dataframes
df_market = pd.read_csv("input/markets.csv")
df_runners = pd.read_csv("input/runners.csv",dtype={'barrier': np.int16,'handicap_weight': np.float16})

#for my simple model, I'm ignoring other columns. I recommend starting with form if you're looking to add features
#df_odds = pd.read_csv("../input/odds.csv")
#df_form = pd.read_csv("../input/forms.csv")
#df_condition = pd.read_csv("../input/conditions.csv")
#df_weather = ("../input/weather.csv")
#df_rider = ("../input/riders.csv")
#df_horse = ("../input/horses.csv")
#df_horse_sex = ("../input/horse_sexes.csv")

In [3]:
##merge the runners and markets data frames
df_runners_and_market = pd.merge(df_runners,df_market,left_on='market_id',right_on='id',how='outer')
#df_runners_and_market.index = df_runners_and_market['id_x'] 
df_runners_and_market = df_runners_and_market.rename(columns={'id_x': 'runner_id'})

In [4]:
df_runners_and_market.head()

,runner_id,collected_at,market_id,position,place_paid,margin,horse_id,trainer_id,rider_id,handicap_weight,number,barrier,blinkers,emergency,form_rating_one,form_rating_two,form_rating_three,last_five_starts,favourite_odds_win,favourite_odds_place,favourite_pool_win,favourite_pool_place,tip_one_win,tip_one_place,tip_two_win,...,tip_four_win,tip_four_place,tip_five_win,tip_five_place,tip_six_win,tip_six_place,tip_seven_win,tip_seven_place,tip_eight_win,tip_eight_place,tip_nine_win,tip_nine_place,id_y,timezone,venue_id,race_number,distance,condition_id,weather_id,total_pool_one_win,total_pool_one_place,total_pool_two_win,total_pool_two_place,total_pool_three_win,total_pool_three_place
0,4.0,2016-06-26 18:54:31.800293,1.0,NaN,1.0,NaN,4.0,4.0,4.0,58.5,4.0,10.0,f,f,82.0,82.0,14.0,x80x2,f,f,f,f,f,NaN,f,...,t,t,t,t,f,t,f,t,f,t,t,t,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
1,10.0,2016-06-26 18:54:31.974395,1.0,NaN,0.0,NaN,10.0,4.0,10.0,56.5,10.0,11.0,f,f,100.0,100.0,18.0,22x35,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
2,5.0,2016-06-26 18:54:31.835329,1.0,NaN,0.0,NaN,5.0,5.0,5.0,56.5,5.0,5.0,f,f,76.0,76.0,0.0,f7,f,f,f,f,f,NaN,f,...,f,f,f,t,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
3,6.0,2016-06-26 18:54:31.873492,1.0,NaN,0.0,NaN,6.0,6.0,6.0,56.5,6.0,12.0,f,f,85.0,85.0,2.0,f6462,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0
4,1.0,2016-06-26 18:54:31.746854,1.0,NaN,0.0,NaN,1.0,1.0,1.0,58.5,1.0,8.0,f,f,89.0,89.0,0.0,34x0x,f,f,f,f,f,NaN,f,...,f,f,f,f,f,f,f,f,f,f,f,f,1.0,2016-06-26 19:00:00,1.0,3.0,1200.0,1.0,1.0,29718.08,NaN,11564.5,5373.0,23464.8,5373.0


In [7]:
numeric_features = ['position', 'horse_id', 'runner_id', 'market_id','barrier','handicap_weight']
categorical_features = []

#convert to factors
'''for feature in categorical_features:
    df_runners_and_market[feature] = df_runners_and_market[feature].astype(str)
    df_runners_and_market[feature] = df_runners_and_market[feature].replace('nan','0') #have to do this because of a weird random forest bug

    df_features = df_runners_and_market[numeric_features]

for feature in categorical_features:
    encoded_features = pd.get_dummies(df_runners_and_market[feature])
    encoded_features.columns = feature + encoded_features.columns
    df_features = pd.merge(df_features,encoded_features,left_index=True,right_index=True,how='inner') 
'''
df_features = df_runners_and_market[numeric_features]
#turn the target variable into a binary feature: did or did not win
df_features['win'] = False
df_features.loc[df_features['position'] == 1,'win'] = True
#df_features = df_features.reset_index().rename(columns={'id_x': 'horse_id'})
df_features = df_features.reindex(np.random.permutation(df_features.index))
#del df_runners_and_market, encoded_features, df_features['position']

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
def shapeToRaces(race):
    
    won = race[race.win == True]
    if won.empty:
        # some races don't have a winner. we'll use the -8888 to remove them later
        wonClass = -8888; 
    else:
        won = won.index.values[0]
        wonClass = race.index.get_loc(won) # get what class the winning horse will be    race =     
    
    market_id = race.iloc[0,:].market_id
    runner_ids = race.runner_id.values    
        
    #drop unneccesary columns before we go to straight numpy arrays
    race = race.drop(['horse_id', 'win','position', 'runner_id', 'market_id'], axis=1)
        
    #stack horse from the race into one flat row
    st = race.stack().to_frame().T #stack it
        
    #now lose the multi-index and extract just values as just a numpy array
    st.columns = ['{}_{}'.format(col[1], col[0]) for col in st.columns]
    vals = st.values
    
    #add fake columns to create features for 25 horses
    colsToAdd = (race.shape[1] * 25) - st.shape[1]
    ones = (np.ones(colsToAdd) * -999)[:, np.newaxis].T
    withAdded = np.concatenate((vals, ones ), axis=1)
        
    #append label and other meta_data to the row
    withAdded = np.append(withAdded, wonClass)
    
    return (withAdded, runner_ids)

In [8]:
m = df_features.groupby('market_id').apply(shapeToRaces)

In [234]:
from shapeRaces import makeRaces
vals, classDf, market_ids = makeRaces(df_features)

ImportError: cannot import name 'makeRaces'

In [177]:
runner_ids = m.apply(lambda x: x[1])

d = pd.DataFrame(runner_ids.reset_index().rename(columns={0: 'runners'}))

def getMarketList(race):
    ids = np.ones(race.runners.shape[0]) * race.market_id
    classes = np.arange(race.runners.shape[0])
    return list(zip(race.runners, classes, ids))
d['market_list'] = d.apply(getMarketList,axis=1)

runnersClasses = np.vstack(d.market_list.values)
classDf = pd.DataFrame({'runner_id':runnersClasses[:,0], 'class':runnersClasses[:,1],'market_id':runnersClasses[:,2]})

vals = m.apply(lambda x: x[0]).values
vals = np.vstack(vals)
valsMask = vals[:, -1] > -8888
vals = vals[valsMask]
market_ids = m.index.values
market_ids = market_ids[valsMask]
print(vals.shape)
print(market_ids.shape)

In [185]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(vals[:, :-1], vals[:, -1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [205]:
probs = lr.predict_proba(vals[:, :-1])
colsToAdd = list(np.arange(probs.shape[1]))
probs = np.hstack((probs, market_ids[:, np.newaxis]))
colsToAdd.append('market_id')

In [209]:
probs_df = pd.DataFrame(probs, columns=colsToAdd)
probs_re = probs_df.set_index('market_id').stack().reset_index().rename(columns={0:'prob', 'level_1':'class'})

classAndProb = probs_re.merge(classDf, on=['market_id', 'class'])    

classAndProb.head()

In [220]:
from runRacesMultiClass import addOdds
c = addOdds(classAndProb, all_odds)

In [224]:
d.head()

,market_id,runners,market_list
0,1.0,"[12.0, 10.0, 9.0, 3.0, 4.0, 13.0, 11.0, 6.0, 5...","[(12.0, 0, 1.0), (10.0, 1, 1.0), (9.0, 2, 1.0)..."
1,2.0,"[19.0, 16.0, 15.0, 18.0, 24.0, 21.0, 22.0, 20....","[(19.0, 0, 2.0), (16.0, 1, 2.0), (15.0, 2, 2.0..."
2,3.0,"[38.0, 36.0, 41.0, 34.0, 39.0, 42.0, 31.0, 32....","[(38.0, 0, 3.0), (36.0, 1, 3.0), (41.0, 2, 3.0..."
3,4.0,"[47.0, 48.0, 51.0, 45.0, 53.0, 44.0, 56.0, 52....","[(47.0, 0, 4.0), (48.0, 1, 4.0), (51.0, 2, 4.0..."
4,5.0,"[62.0, 65.0, 61.0, 67.0, 59.0, 58.0, 70.0, 60....","[(62.0, 0, 5.0), (65.0, 1, 5.0), (61.0, 2, 5.0..."


In [225]:
e = c.merge(df_features[['runner_id', 'win']], on='runner_id')

In [232]:
from runRacesMultiClass import placeBets
placeBets(e)

Total bets: 32101
Total wins: 1400


-12.69

In [238]:
from shapeRaces import makeRaces
raceData, runnerClassDf, market_ids = makeRaces(df_features)

In [252]:
from sklearn.cross_validation import train_test_split
train, test, train_market_ids, test_market_ids = train_test_split(raceData, market_ids)

In [257]:
test.shape

(1592, 51)

In [253]:
train.shape

(4773, 51)

In [254]:
train_market_ids.shape

(4773,)

In [256]:
test_market_ids.shape

(1592,)

In [81]:
pd.DataFrame(raceData).to_csv('raceDataBasic.csv', header=True, index=False)
runnerClassDf.to_csv('runnerClassBasic.csv', header=True, index=False)


In [45]:
runnerClassDf.head()

,class,horse_id,market_id,runner_id
197,0.0,436.0,16.0,446.0
198,1.0,262.0,16.0,445.0
196,2.0,432.0,16.0,441.0
195,3.0,435.0,16.0,444.0
199,4.0,433.0,16.0,442.0


In [ ]:
ru

In [71]:
from runRacesMultiClass import trainTestSplit
train, test = trainTestSplit(raceData)
print(train.shape)
test.shape

(4456, 53)


(1909, 53)

In [219]:
all_odds = pd.read_csv("input/odds.csv")

In [261]:
from runMultiFaster import runRaces
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
runRaces(data=df_features, est=lr, allOdds= all_odds, n_trials=3)


reshaping data
trial 0
fitting estimator
making predictions
Total bets: 8024
Total wins: 304
trial 1
fitting estimator
making predictions
Total bets: 7874
Total wins: 317
trial 2
fitting estimator
making predictions
Total bets: 8101
Total wins: 330
mean payout: -17.816666666666666, std payout: 3.0791160347663986


In [218]:
classAndProb.head()

,market_id,class,prob,runner_id
0,16.0,0,0.235044,442.0
1,16.0,1,0.146341,445.0
2,16.0,2,0.107188,444.0
3,16.0,3,0.156596,446.0
4,16.0,4,0.222932,441.0


In [85]:
from runRacesMultiClass import addOdds
classAndProb = addOdds(classAndProb, all_odds)

In [86]:
classAndProb.head()

,market_id,class,prob,horse_id,runner_id,odds
0,23.0,0.0,0.055860,266.0,268.0,11.88
1,23.0,1.0,0.141468,263.0,264.0,23.30
2,23.0,2.0,0.129215,267.0,269.0,35.80
3,23.0,3.0,0.062784,265.0,266.0,11.90
4,23.0,5.0,0.044976,270.0,273.0,10.31


In [88]:
from runRacesMultiClass import placeBets
classAndProb = classAndProb.merge(df_features[['runner_id', 'win']], on='runner_id')
placeBets(classAndProb)

Total bets: 9642
Total wins: 367


-20.38

In [235]:
%reload_ext autoreload

In [78]:
probs = lr.predict_proba(X)

market_ids = X_test[:,-1].T

probs_df = pd.DataFrame()
probs_re = probs_df.set_index('market_id').stack().reset_index().rename(columns={0:'prob', 'level_1':'class'})

probs_re['class'] = probs_re['class'].astype(float)

classAndProb = probs_re.merge(runnerClassDf, on=['market_id', 'class'])

In [187]:
classAndProb = classAndProb.merge(df_features[['runner_id', 'win']], on='runner_id')